In [1]:
import numpy as np
import pandas as pd

In [2]:
dat = pd.read_csv("bgg_dataset_raw.csv", delimiter=";").dropna().reset_index()
dat.shape

(9709, 15)

In [3]:
dat.columns = dat.columns.str.lower().str.replace(" ", "_", regex=True)
dat = dat.astype({
    "id": "int64",
    "year_published": "int64",
    "owned_users": "int64",
})
dat["rating_average"] = dat["rating_average"].str.replace(",", ".").astype(float)
dat["complexity_average"] = dat["complexity_average"].str.replace(",", ".").astype(float)

domains_split = dat["domains"].str.split(", ", expand=True)
domains_split.columns = [f"domains{i+1}" for i in range(domains_split.shape[1])]

dat = pd.concat([dat, domains_split], axis=1)

dat = dat.drop(['mechanics', 'domains2', 'domains3', 'domains', 'bgg_rank', 'users_rated'], axis=1)

In [4]:
dat = dat[(dat["max_players"] <= 30) & 
          (dat["max_players"] > 0) & 
          (dat["min_players"] > 0) & 
          (dat["min_players"] <= dat["max_players"])]

print(dat.shape)

dat = dat[(dat["name"].str.count("-") <= 1) & 
          (dat["name"].str.count("&") <= 1) & 
          (dat["name"].str.split().str.len() <= 10)]

print(dat.shape)

dat = dat[(dat["play_time"] > 0) & 
          (dat["play_time"] <= 360)]

print(dat.shape)

dat = dat[(dat["year_published"] > 0)]

print(dat.shape)

(9597, 12)
(9396, 12)
(8965, 12)
(8917, 12)


In [5]:
dat

,index,id,name,year_published,min_players,max_players,play_time,min_age,rating_average,complexity_average,owned_users,domains1
0,0,174430,Gloomhaven,2017,1,4,120,14,8.79,3.86,68323,Strategy Games
1,1,161936,Pandemic Legacy: Season 1,2015,2,4,60,13,8.61,2.84,65294,Strategy Games
2,2,224517,Brass: Birmingham,2018,2,4,120,14,8.66,3.91,28785,Strategy Games
3,3,167791,Terraforming Mars,2016,1,5,120,12,8.43,3.24,87099,Strategy Games
5,5,291457,Gloomhaven: Jaws of the Lion,2020,1,4,120,14,8.87,3.55,21609,Strategy Games
...,...,...,...,...,...,...,...,...,...,...,...,...
9701,20334,3522,LCR,1983,3,12,20,5,3.42,1.05,3441,Children's Games
9702,20335,1406,Monopoly,1933,2,8,180,8,4.39,1.65,40255,Family Games
9703,20336,2921,The Game of Life,1960,2,6,60,8,4.30,1.18,16692,Family Games
9704,20337,1410,Trouble,1965,2,4,45,4,3.79,1.05,4962,Children's Games


In [6]:
dat.columns

Index(['index', 'id', 'name', 'year_published', 'min_players', 'max_players',
       'play_time', 'min_age', 'rating_average', 'complexity_average',
       'owned_users', 'domains1'],
      dtype='object')

In [7]:
dat.to_json("bgg_dataset_cleaned.json", orient="records", indent=4)
dat.to_json("src/bgg_dataset_cleaned.json", orient="records", indent=4)

In [8]:
dat[dat["name"].str.len() == max(dat["name"].str.len())]

,index,id,name,year_published,min_players,max_players,play_time,min_age,rating_average,complexity_average,owned_users,domains1
4331,5278,120265,Supply Lines of the American Revolution: The N...,2017,2,2,180,0,7.59,2.7,530,Wargames
